<a href="https://colab.research.google.com/github/smf-9000/kaggle/blob/main/%5B003_1%5D_tps_aug_2021.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

```
I'm trying to push lightGBM to the end. All in all, I think that "fold" concept has some potential.
```

In [ ]:
%config Completer.use_jedi = False

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns

import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow, imread

from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold

import lightgbm as lgb
import warnings

In [ ]:
submission_ex = pd.read_csv('/kaggle/input/tabular-playground-series-aug-2021/sample_submission.csv')
train_df = pd.read_csv('/kaggle/input/tabular-playground-series-aug-2021/train.csv')
test_df = pd.read_csv('/kaggle/input/tabular-playground-series-aug-2021/test.csv')

In [ ]:
targets_df = train_df[['loss']].copy()
train_df.drop(['id', 'loss'], axis=1, inplace=True) 
test_df.drop(['id'], axis=1, inplace=True) 

In [ ]:
def softmax(x):
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum(axis = 0)

In [ ]:
params_loss = {'bagging_fraction': 0.8962250586874196, 
               'bagging_freq': 1, 
               'lambda_l1': 0.01895614209401383, 
               'lambda_l2': 0.001397920954070796, 
               'learning_rate': 0.05906516799338329, 
               'max_depth': 46, 
               'min_sum_hessian_in_leaf': 376.9636549547788, 
               'n_estimators': 100, 
               'num_leaves': 184}


params_loss['n_estimators'] = 20000
params_loss['learning_rate'] = 0.005

lgbm_reg = lgb.LGBMRegressor(**params_loss, 
                       objective='rmse',
                       metric='rmse',
                       n_jobs=-1)

In [ ]:
submit = True

```
300 epochs

fold 2, 4, 6, 8, 10
===================
all_rmse:  7.883865767912834
end_rmse1:  7.91782919532188
end_rmse2:  7.917809927969676	###
end_rmse3:  7.917848576969314

fold 10
===================
all_rmse:  7.883768205859681
end_rmse1:  7.918339059278837
end_rmse2:  7.918326740043123	###
end_rmse3:  7.918351779650094

fold 20
===================
all_rmse:  7.883478431519637
end_rmse1:  7.918303796204797
end_rmse2:  7.918301322304228	###
end_rmse3:  7.918306860761374
```

In [ ]:
_target = 'loss'

if submit:
    X = train_df
    y = targets_df[['loss']]
else:
    np.random.seed(37)
    msk = np.random.rand(len(train_df)) < 0.9
    X = train_df[msk]
    my_X = train_df[~msk]
    y = targets_df[msk]
    my_y = targets_df[~msk]

pred = []
val_rmse = []
random_state = 37
all_rmse = 0

print(X.shape)

for k in range(2, 11, 2):
    print('k: ', k)
    print('====================')
    kfolds = KFold(n_splits = k, shuffle = True, random_state = random_state)
    k_folds_rmse = 0
    _k = 1
    for train_index, test_index in kfolds.split(X):
        print('------------------------------------------------k: ', _k, 'of', k)
        _k += 1
        X_train, X_val = X.iloc[train_index], X.iloc[test_index]
        y_train, y_val = y.iloc[train_index], y.iloc[test_index]
        
        lgbm_reg.fit(
            X_train, 
            y_train,
            eval_set=[(X_val, y_val)],
            verbose=500,
            early_stopping_rounds=500)
        
        f_pred = lgbm_reg.predict(X_val)
        curr_rmse = mean_squared_error(y_val[_target].values, f_pred, squared = False)
        print('curr_rmse: ', curr_rmse)
        val_rmse.append(curr_rmse)
        
        all_rmse += curr_rmse
        k_folds_rmse += curr_rmse
        
        pred.append(lgbm_reg.predict(my_X if not submit else test_df))
        
    print('k_folds_rmse: ', k_folds_rmse / k)
        
    print('all_rmse: ', all_rmse / len(pred))

    
pred1 = np.sum(pred, axis = 0) / len(pred)
pred2 = np.transpose(np.matmul(np.transpose(pred), softmax(val_rmse)))
pred3 = np.transpose(np.matmul(np.transpose(pred), softmax(np.squeeze(np.full((1, len(pred)), 100)) - val_rmse)))

if not submit:
    end_rmse1 = mean_squared_error(my_y, pred1, squared = False)
    print('end_rmse1: ', end_rmse1)
    end_rmse2 = mean_squared_error(my_y, pred2, squared = False)
    print('end_rmse2: ', end_rmse2)
    end_rmse3 = mean_squared_error(my_y, pred3, squared = False)
    print('end_rmse3: ', end_rmse3)
else:
    submission_1 = submission_ex[['id']].copy()
    submission_2 = submission_ex[['id']].copy()
    submission_3 = submission_ex[['id']].copy()
    submission_1[_target] = pred1
    submission_2[_target] = pred2
    submission_3[_target] = pred3
    submission_1.to_csv('submission_1.csv', index=False)
    submission_2.to_csv('submission_2_p.csv', index=False)
    submission_3.to_csv('submission_3.csv', index=False)

In [ ]:
test = pd.read_csv('submission_2_p.csv')
test.head()